In [2]:
import pandas as pd

X_df = pd.read_feather('./preprocessed_data/x.feather')
y = pd.read_csv('./preprocessed_data/y.csv')
y = y['score']

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, test_size=0.2, stratify=y
)
X_train = X_train.dropna()
X_test = X_test.dropna()

import random

sgd = SGDClassifier(
    loss='log_loss',
    warm_start=True,
    n_jobs=-1
)
# sgd.partial_fit(X_train, y_train, classes=[0,1])
for k in range(100):
    sgd.partial_fit(X_train.sample(20), y_train.sample(20), classes=[0,1,2])

y_pred = sgd.predict(X_test)
accuracy_score(y_test, y_pred)

0.27007299270072993

In [17]:
from LeIA import SentimentIntensityAnalyzer

s = SentimentIntensityAnalyzer()

In [3]:
# from bert_sklearn import BertClassifier

# bert = BertClassifier()
# bert.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# accuracy_score(y_test, y_pred)

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint


/home/pedro/Desktop/sentimental_vasco/.venv/lib/python3.12/site-packages/bert_sklearn/model/pytorch_pretrained/modeling.py:755: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

train data size: 490, validation data size: 54


Training  :   0%|          | 0/16 [00:00<?, ?it/s]